In [ ]:
import sys
sys.path.append("..")

import random
import math
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.cnn import *

In [ ]:
SHAPE = (1, 32, 32)
CODE_SIZE = 64
dataset = TensorDataset(torch.load(f"../datasets/fonts-regular-{SHAPE[-2]}x{SHAPE[-1]}.pt"))
#dataset = TransformDataset(dataset, dtype=torch.float, multiply=255.)
assert SHAPE == dataset[0][0].shape
model = ConvAutoEncoder(SHAPE, channels=[8, 16, 24], kernel_size=7, code_size=CODE_SIZE)
model.load_state_dict(torch.load("../checkpoints/font1/snapshot.pt")["state_dict"])

In [ ]:
SHAPE = (3, 64, 64)
CODE_SIZE = 64
model = ConvAutoEncoder(SHAPE, channels=[32, 32, 23], code_size=CODE_SIZE)
model.load_state_dict(torch.load("../models/ae/kali-cnn/snapshot.pt")["state_dict"])

In [ ]:
dataset = TensorDataset(torch.load(f"../datasets/kali-uint8-{SHAPE[-2]}x{SHAPE[-1]}.pt"))
dataset = TransformDataset(dataset, dtype=torch.float, multiply=1./255.)

In [ ]:
images = torch.cat([dataset[i][0].unsqueeze(0) for i in RandomSampler(dataset, num_samples=16)])
repros = model.forward(images).clip(0, 1)
VF.to_pil_image(make_grid(torch.cat([images, repros]), nrow=16))

# dataset -> features

In [ ]:
def encode_dataset(dataset):
    feature_list = []
    for image_batch, in tqdm(DataLoader(dataset, batch_size=50)):
        features = model.encoder(image_batch)
        feature_list.append(features)
        #if len(feature_list) >= 100:
        #    break
    return torch.cat(feature_list)


with torch.no_grad():
    features = encode_dataset(dataset)
print(features.shape)
VF.to_pil_image(features[:100])

# PCA of features

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA(CODE_SIZE)
pca.fit(features)
#pca.components_
pca_weight = torch.Tensor(pca.components_)
pca_features = features @ pca_weight.T
VF.to_pil_image(pca_weight)

In [ ]:
px.line(pca_features.std(0))

In [ ]:
print(features[0])
print(torch.round(features[0] - (pca_features[0] @ pca_weight), decimals=2))

# generate from PCA features

In [ ]:
def plot_pca_samples(features, nrow=16):
    features = features @ pca_weight
    repros = model.decode(features).clip(0, 1)
    
    display(VF.to_pil_image(make_grid(repros, nrow=nrow)))
    
plot_pca_samples(pca_features[:32])

# blend

In [ ]:
def blend_feature_plot(sample1, samples2, nrow=16):
    samples = []
    for sample2 in samples2:
        for i in range(nrow):
            t = i / (nrow - 1)
            samples.append( (sample1 * (1. - t) + t * sample2).unsqueeze(0) )

    features = torch.cat(samples) @ pca_weight
    repros = model.decode(features).clip(0, 1)#.view(-1, *SHAPE)
    display(VF.to_pil_image(make_grid(repros, nrow=nrow)))
    
blend_feature_plot(pca_features[4], pca_features[18000:18010])

# gen from PCA features

In [ ]:
def plot_pca_sample(pca_sample, nrow=16, comps=5):
    modified_samples = pca_sample.view(1, -1).repeat(nrow * comps, 1)
    for i, sample in enumerate(modified_samples):
        t = ((i % nrow) / 15.) * 2. - 1.
        idx = i // nrow
        sample[idx] = t * 7.
    samples = modified_samples @ pca_weight
    repros = model.decoder(samples).clip(0, 1)
    display(VF.to_pil_image(make_grid(repros, nrow=nrow)))
    
plot_pca_sample(pca_features[11007])

In [ ]:
def plot_samples(
        iterable, 
        total: int = 16, 
        nrow: int = 16, 
        return_image: bool = False, 
):
    samples = []
    f = ImageFilter()
    try:
        for image in tqdm(iterable, total=total):
            samples.append(image)
                
            if len(samples) >= total:
                break
    except KeyboardInterrupt:
        pass
    
    image = VF.to_pil_image(make_grid(samples, nrow=nrow))
    if return_image:
        return image
    display(image)
    
plot_samples(
    ()
)